In [ ]:
from __future__ import division

In [ ]:
import os
from pathlib import Path
import string
import nltk
nltk.download('stopwords')
import docx
import pytesseract
from PIL import Image
# https://stackoverflow.com/questions/22676/how-to-download-a-file-over-http
import urllib.request
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file
# https://stackoverflow.com/questions/2693820/extract-images-from-pdf-without-resampling-in-python
import fitz  # pip install --upgrade pip; pip install --upgrade pymupdf
from tqdm import tqdm # pip install tqdm

from typing_extensions import ParamSpecArgs
import fitz # install using: pip install PyMuPDF
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import shutil

from struct import pack, unpack

import binascii

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
workspace_folder_path = None
while True:
  print("Mohon masukkan alamat folder yang dapat digunakan oleh program ini " + \
      "untuk beroperasi: ")
  # Folder yang digunakan oleh software untuk beroperasi
  workspace_folder_path = str(input())
  if not os.path.exists(workspace_folder_path):
    print("Mohon maaf, folder tidak tersedia atau alamat folder tidak valid, mohon masukkan kembali alamat folder")
  elif not os.path.isdir(workspace_folder_path):
    print("Mohon maaf, alamat folder tidak valid, mohon masukkan kembali alamat folder")
  else:
    print("Alamat folder berhasil disimpan")
    break

Mohon masukkan alamat folder yang dapat digunakan oleh program ini untuk beroperasi: 
/
Alamat folder berhasil disimpan


In [ ]:
indexing_folder = os.path.join(workspace_folder_path, "indexing_folder")
indexing_file = os.path.join(indexing_folder, "index.txt")
indexing_document_path_to_document_id_file = os.path.join(indexing_folder, "document_id.txt")
indexing_ulang = None
if os.path.exists(indexing_file) and os.path.exists(indexing_document_path_to_document_id_file):
  while True:
    print("Telah ditemukan hasil indexing sebelumnya. Ketik Y untuk" + " " +
          "menggunakan hasil indexing sebelumnya, ketik N untuk" + " " +
          "melakukan indexing ulang. Jika melakukan indexing ulang" + " " + 
          "maka hasil indexing sebelumnya akan dihapus.")
    use_previous_indexing = str(input()).lower()
    if len(use_previous_indexing) > 1 or len(use_previous_indexing) < 1 or \
    (use_previous_indexing.lower() != "y" and \
    use_previous_indexing.lower() != "n"):
      print("Mohon ketik Y atau N.")
    elif use_previous_indexing.lower() == "y":
      while True:
        print("Apakah anda yakin ingin menggunakan hasil indexing sebelumnya?"+
              " " + "(Y/N)")
        use_previous_indexing = str(input())
        if len(use_previous_indexing) > 1 or len(use_previous_indexing) < 1 or \
        (use_previous_indexing.lower() != "y" and \
        use_previous_indexing.lower() != "n"):
          print("Mohon ketik Y atau N.")
        elif use_previous_indexing.lower() == "y":
          print("Akan menggunakan hasil indexing sebelumnya.")
          indexing_ulang = False
          break
        elif use_previous_indexing.lower() == "n":
          break
        else:
          raise ValueError("Telah terjadi error.")
    elif use_previous_indexing.lower() == "n":
      while True:
        print("Apakah anda yakin ingin melakukan indexing ulang dan"+
              " " + "menghapus hasil indexing sebelumya?" + " " +
              "(Y/N)")
        use_previous_indexing = str(input())
        if len(use_previous_indexing) > 1 or len(use_previous_indexing) < 1 or \
        (use_previous_indexing.lower() != "y" and \
        use_previous_indexing.lower() != "n"):
          print("Mohon ketik Y atau N.")
        elif use_previous_indexing.lower() == "y":
          print("Akan melakukan indexing ulang dan menghapus hasil indexing"+
                " " + "sebelumnya")
          indexing_ulang = True
          break
        elif use_previous_indexing.lower() == "n":
          break
        else:
          raise ValueError("Telah terjadi error.")
    else:
      raise ValueError("Telah terjadi error")
    if indexing_ulang is not None:
      break
else:
  indexing_ulang = True


Telah ditemukan hasil indexing sebelumnya. Ketik Y untuk menggunakan hasil indexing sebelumnya, ketik N untuk melakukan indexing ulang. Jika melakukan indexing ulang maka hasil indexing sebelumnya akan dihapus.
y
Apakah anda yakin ingin menggunakan hasil indexing sebelumnya? (Y/N)
y
Akan menggunakan hasil indexing sebelumnya.


In [ ]:
# VBCode
def vbcode_encode_number(number):
    """Variable byte code encode number.
    Usage:
      import vbcode
      vbcode.encode_number(128)
    """
    bytes_list = []
    while True:
        bytes_list.insert(0, number % 128)
        if number < 128:
            break
        number = number // 128
    bytes_list[-1] += 128
    return pack('%dB' % len(bytes_list), *bytes_list)

def vbcode_encode(numbers):
    """Variable byte code encode numbers.
    Usage:
      import vbcode
      vbcode.encode([32, 64, 128])
    """
    bytes_list = []
    for number in numbers:
        bytes_list.append(vbcode_encode_number(number))
    return b"".join(bytes_list)

def vbcode_decode(bytestream):
    """Variable byte code decode.
    Usage:
      import vbcode
      vbcode.decode(bytestream)
        -> [32, 64, 128]
    """
    n = 0
    numbers = []
    bytestream = unpack('%dB' % len(bytestream), bytestream)
    for byte in bytestream:
        if byte < 128:
            n = 128 * n + byte
        else:
            n = 128 * n + (byte - 128)
            numbers.append(n)
            n = 0
    return numbers

In [ ]:
def vbcode_bytestream_to_string(bytestream):
  return binascii.unhexlify(bytestream.hex()).decode()

In [ ]:
def vbcode_string_to_bytestream(string):
  return bytes.fromhex(binascii.hexlify(string)).encode()


In [ ]:
# SPIMI

# terms adalah array 2 dimensi
# terms[0] adalah string
# terms[1] adalah integer

output_directory = indexing_folder

block_prefix = "block"
block_suffix = ".txt"

output_index = indexing_file
output_document_id = indexing_document_path_to_document_id_file

dictionary = {}
# Jika value nya di set maka akan menulis block ke file secara otomatis jika
# dictionary sudah penuh
dictionary_max_key = [1000]
dictionary_max_item = [50000]
dictionary_key_count = [0]
dictionary_item_count = [0]
highest_document_id = [0]
document_path_to_document_id = {}
block_files = []
block_number = [0]

# fungsi yang diawali dengan underscore "_" adalah fungsi privat
# artinya tidak boleh dipanggil oleh proses luar

def _add_to_dictionary(dictionary, term, dictionary_key_count=dictionary_key_count):
  if term not in dictionary:
    dictionary_key_count[0] += 1
    dictionary[term] = []
  return dictionary[term]

def _get_postings_list(dictionary, term):
  return dictionary[term]

def _add_to_postings_list(postings_list, document_path, dictionary_item_count=dictionary_item_count):
  dictionary_item_count[0] += 1
  postings_list.append(document_path)

def _sort_terms(dictionary):
  return [term for term in sorted(dictionary)]

def write_block_to_output_directory(block_number=block_number, \
                                    dictionary=dictionary, \
                                    block_prefix=block_prefix, \
                                    block_suffix=block_suffix, \
                                    dictionary_key_count=dictionary_key_count, \
                                    dictionary_item_count=dictionary_item_count, \
                                    output_directory=output_directory, \
                                    document_path_to_document_id=document_path_to_document_id):
  block_number[0] += 1
  sorted_terms = _sort_terms(dictionary)

  block_file = os.path.join(output_directory, "".join([block_prefix, 
                                                        str(block_number[0]), 
                                                        block_suffix]))
  with open(block_file, 'w') as file:
      for term in sorted_terms:
          line = "%s %s\n" % (term, ' '.join([str(document_id) for document_id in dictionary[term]]))
          file.write(line)
  block_files.append(block_file)
  # reset dictionary setelah menulis ke file block
  dictionary = {}
  dictionary_key_count[0] = 0
  dictionary_item_count[0] = 0

# token adalah array-like dengan 2 data
# token[0] adalah term nya
# token[1] adalah document path
def add_terms_to_block(token, dictionary_max_key=dictionary_max_key, \
                       dictionary_max_item=dictionary_max_item, \
                       dictionary_key_count=dictionary_key_count, \
                       dictionary_item_count=dictionary_item_count, \
                       highest_document_id=highest_document_id, \
                       document_path_to_document_id=document_path_to_document_id):
    try:
      if token[0] and token[1]:
        pass
    except:
      raise ValueError("Error: token tidak berbentuk array, namun %s" % \
                       type(token))
    if len(token) < 2:
      raise ValueError("Error: jumlah elemen pada token kurang dari 2")
    if not os.path.exists(token[1]):
      raise ValueError("Error: file dokumen pada token[1] tidak ditemukan ")
    if not os.path.isfile(token[1]):
      raise ValueError("Error: path pada token[1] mengarah ke folder" + " "+
                       "bukan file")


    if dictionary_max_key is not None and dictionary_max_item is not None:
      if dictionary_max_key[0] <= 0 or dictionary_max_item[0] <= 0:
        raise ValueError("Error: dictionary_max_key dan dictionary_max_item"+
                         " " + "tidak boleh kurang dari atau sama dengan 0")
      # jika ukuran dictionary melebihi batas, maka tulis dictionary ke file
      if dictionary_key_count[0] >= dictionary_max_key[0] or \
      dictionary_item_count[0] >= dictionary_max_item[0]:
        write_block_to_output_directory()
    if token[1] not in document_path_to_document_id:
      highest_document_id[0] += 1
      document_path_to_document_id[token[1]] = highest_document_id[0]
    terms = [token[0], document_path_to_document_id[token[1]]]
    postings_list = _add_to_dictionary(dictionary, terms[0])
    _add_to_postings_list(postings_list, terms[1])

def construct_index():
  write_block_to_output_directory()
  return _merge_blocks(block_files)

def _merge_blocks(block_files, output_index=output_index, document_path_to_document_id=document_path_to_document_id):
  block_files = [open(block_file) for block_file in block_files]
  lines = [block_file.readline()[:-1] for block_file in block_files]
  most_recent_term = ""

  index = 0
  for block_file in block_files:
      if lines[index] == "":
          block_files.pop(index)
          lines.pop(index)
      else:
          index += 1

  with open(output_index, "w") as output_index:
      while len(block_files) > 0:

          min_index = lines.index(min(lines))
          line = lines[min_index]
          current_term = line.split()[0]
          current_postings = " ".join(map(str, sorted(list(map(int, line.split()[1:])))))

          if current_term != most_recent_term:
              output_index.write("\n%s %s" % (current_term, current_postings))
              most_recent_term = current_term
          else:
              output_index.write(" %s" % current_postings)

          lines[min_index] = block_files[min_index].readline()[:-1]

          if lines[min_index] == "":
              block_files[min_index].close()
              block_files.pop(min_index)
              lines.pop(min_index)

      output_index.close()
  with open(indexing_document_path_to_document_id_file, 'w') as file:
      for document_path in document_path_to_document_id:
          line1 = "%s\n" % document_path
          line2 = "%s\n" % str(document_path_to_document_id[document_path])
          file.write(line1)
          file.write(line2)
  file.close()

  return get_index()

def get_index(index_file=indexing_file, \
               document_id_file=indexing_document_path_to_document_id_file):
  inverted_index = {}
  document_id_and_path = {}

  index_file = open(output_index)
  index_file.readline()

  for line in index_file:
      line = line.split()
      inverted_index[line[0]] = sorted(map(int, (line[1:])))

  document_id_file = open(document_id_file)
  line0 = None
  line1 = None
  for line in document_id_file:
      # line[0] = document path
      # line[1] = document id
      # document id => document path
      if line0 is None:
        line0 = line
        line1 = None
      elif line1 is None:
        line1 = line
        document_id_and_path[int(line1)] = line0.rstrip()
        line0 = None
        line1 = None
  document_id_file.close()
  
  

  return (inverted_index, document_id_and_path)

In [ ]:
# print(lines)

In [ ]:
def text_transformation(word):
  ps = PorterStemmer()
  stop_words = stopwords.words('english')
  # remove punctuation
  word = word.translate(str.maketrans('', '', string.punctuation))
  # stopword removal
  if word not in stop_words:
    word = ps.stem(word)
  
  return word

In [ ]:
if indexing_ulang:
  list_of_path = []
  if not os.path.exists(indexing_folder):
    # buat folder
    os.makedirs(indexing_folder)
  else:
    # hapus folder dan isinya lalu buat foldernya lagi
    shutil.rmtree(indexing_folder)
    os.makedirs(indexing_folder)

  while True:
    print("Masukkan alamat folder/file yang ingin dimasukkan ke mesin pencari")
    path_to_append = input()
    if not os.path.exists(path_to_append):
      print("Alamat folder/file tidak ditemukan, mohon masukkan kembali" +
            " " + "alamat folder/file")
    else:
      print("Alamat folder/file berhasil disimpan")
      list_of_path.append(path_to_append)
      break
  continue_adding_path = True
  while continue_adding_path:
    while True:
      print("Apakah anda ingin memasukkan alamat folder/file lain yang ingin"+
          " " + "ditambah ke mesin pencari? (Y/N)")
      add_more_file_path = str(input()).lower()
      if len(add_more_file_path) > 1 or len(add_more_file_path) < 1 or \
      (add_more_file_path.lower() != "y" and \
      add_more_file_path.lower() != "n"):
        print("Mohon ketik Y atau N.")
      elif add_more_file_path.lower() == "y":
        continue_adding_path = True
        break
      elif add_more_file_path.lower() == "n":
        continue_adding_path = False
        break
    if not continue_adding_path:
      break
    print("Masukkan alamat folder/file yang ingin dimasukkan ke mesin pencari")
    path_to_append = input()
    if not os.path.exists(path_to_append):
      print("Alamat folder/file tidak ditemukan, mohon masukkan kembali" +
            " " + "alamat folder/file")
    else:
      print("Alamat folder/file berhasil disimpan")
      list_of_path.append(path_to_append)
  
  print("Sedang melakukan proses indexing, harap tunggu...")

  def send_text_to_spimi(list_of_text, file_path):
    for i in range(len(list_of_text)):
      text_to_process = list_of_text[i].split()
      # Text Transformation
      for j in range(len(text_to_process)):
        word_to_process = text_to_process[j]

        word_to_process = text_transformation(word_to_process)
        
        add_terms_to_block((word_to_process, file_path))
  
  def scrape_file(file_path):
    if not os.path.exists(file_path) and not os.path.isdir(file_path):
          raise ValueError("File does not exist: %s" % str(file_path))
    file_extension = Path(file_path).suffix
    if file_extension == ".png" or file_extension == ".jpg" or file_extension == ".jpeg":
        text_image = pytesseract.image_to_string(Image.open(file_path))
        send_text_to_spimi([text_image], file_path)
    elif file_extension == ".txt":
      with open(file_path) as f:
        for line in f.readlines():
          send_text_to_spimi([line], file_path)
      f.close()
    elif file_extension == ".doc":
      # Belum implementasi
      pass
    elif file_extension == ".docx":
      # Belum implementasi OCR
      doc = docx.Document(file_path)
      for para in doc.paragraphs:
        send_text_to_spimi([para.text], file_path)
    elif file_extension == ".pdf":
      # Belum Implementasi OCR
      with fitz.open(file_path) as doc:
          for page in doc:
              send_text_to_spimi([page.get_text("text")], file_path)
    elif file_extension == ".sgm":
      # Belum Implementasi OCR
      with open(file_path) as f:
        for line in f.readlines():
          send_text_to_spimi([line], file_path)
    else:
      # skip karena format file tidak disupport
      pass
  
  current_layer_recursion_stack = list_of_path
  next_layer_recursion_stack = []

  while len(current_layer_recursion_stack) > 0:
    path_to_process = current_layer_recursion_stack.pop()
    if not os.path.exists(path_to_process):
      print("Path does not exists, skipping path \"%s\"" % str(path_to_process))
    elif os.path.isfile(path_to_process):
      print("Scrapping file \"%s\"" % str(path_to_process))
      scrape_file(path_to_process)
    # directory
    else:
      current_folder = path_to_process
      for name in os.listdir(current_folder):
        new_path = os.path.join(current_folder, name)
        if os.path.isfile(new_path):
          print("Scrapping file \"%s\"" % str(new_path))
          scrape_file(new_path)
        else:
          next_layer_recursion_stack.append(new_path)
    if len(current_layer_recursion_stack) <= 0:
        current_layer_recursion_stack = next_layer_recursion_stack
        next_layer_recursion_stack = []
  construct_index()
  print("Proses indexing selesai!")



In [ ]:
print("Sedang loading hasil indexing, harap tunggu...")
indexing_result = get_index()
print("Hasil indexing berhasil di load!")

Sedang loading hasil indexing, harap tunggu...
Hasil indexing berhasil di load!


In [ ]:
dictionary

{}

In [ ]:
indexing_result[0]

{'1': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
 

In [ ]:
indexing_result[1]

{1: 'foldersatu/filelima.pdf',
 2: 'foldersatu/filesatu.txt',
 3: 'foldersatu/Contoh dokumen/cat-descriptions_120396.txt',
 4: 'foldersatu/Contoh dokumen/all-places-strings.lc.txt',
 5: 'foldersatu/Contoh dokumen/all-orgs-strings.lc.txt',
 6: 'foldersatu/Contoh dokumen/feldman-cia-worldfactbook-data.txt',
 7: 'foldersatu/Contoh dokumen/all-people-strings.lc.txt',
 8: 'foldersatu/Contoh dokumen/all-topics-strings.lc.txt',
 9: 'foldersatu/Contoh dokumen/all-exchanges-strings.lc.txt',
 10: 'foldersatu/Contoh dokumen/reut2-000.sgm'}